# 1. Import Data

## Import dependencies, set up environment

1. `python3 -m venv .venv`
2. utilize virtual environment
    - (LINUX/MAC) `source .venv/bin/activate`
    - (WINDOWS) `.venv\Scripts\Activate.    ps1`
3. `pip install -r requirements.txt`

In [1]:
# Jupyter magic
%run ../util/dependencies.py

## FAA Service Discrepancy Reports

In [5]:
filename_1 = '../data/SDR 20260101-20260109.xls'
filename_2 = '../data/SDR 20260110-20260120.xls'
filename_3 = '../data/SDR 20260121-20260131.xls'
filename_4 = '../data/SDR 20260201-20260210.xls'

df = pd.read_html(filename_1)[0]
# df.concat(pd.read_excel(filename_2, engine='xlrd'), ignore_index=True)
# df.concat(pd.read_excel(filename_3, engine='xlrd'), ignore_index=True)
# df.concat(pd.read_excel(filename_4, engine='xlrd'), ignore_index=True)

ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.